########################################################
# EXPERIMENT CODE
########################################################

In [12]:
import os
import getpass
import platform
import sys
import uuid
from datetime import datetime

def prompt_if_none(env_key, prompt_text, default_value="unknown"):
    val = os.getenv(env_key)
    if not val:
        try:
            val = input(f"{prompt_text} (default: {default_value}): ").strip() or default_value
        except Exception:
            val = default_value
    return val

def collect_session_metadata(
    prompt_fields=True,
    fixed_role=None,
    fixed_project_id=None
):
    session_id = str(uuid.uuid4())
    
    session_metadata = {
        "session_id": session_id,
        "username": os.getenv("JUPYTERHUB_USER", getpass.getuser()),
        "timestamp_utc": datetime.utcnow().isoformat(),
        "hostname": platform.node(),
        "platform": platform.system(),
        "os_version": platform.version(),
        "python_version": sys.version.split()[0],
    }

    # Prompt or use defaults
    session_metadata["role"] = fixed_role or (
        prompt_if_none("RESEARCHER_ROLE", "Enter your role", "collaborator") if prompt_fields 
        else os.getenv("RESEARCHER_ROLE", "researcher")
    )
    session_metadata["project_id"] = fixed_project_id or (
        prompt_if_none("PROJECT_ID", "Enter project ID", "default_project") if prompt_fields 
        else os.getenv("PROJECT_ID", "default_project")
    )

    print("\n📌 Session Metadata:")
    for k, v in session_metadata.items():
        print(f"  {k}: {v}")

    return session_metadata


LIBRARY IMPORTS:

In [13]:
# ============================
# 📦 Standard Library Imports
# ============================
import os
import glob
import io
import json
import time
import ast
import pickle
import platform
import subprocess
from datetime import datetime, timezone
from pprint import pprint
from typing import List, Dict, Any
import xml.etree.ElementTree as ET
import urllib.parse
import yaml

# ============================
# 📊 Data and Visualization
# ============================
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

# ============================
# 🤖 Machine Learning
# ============================
import sklearn
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, label_binarize
from sklearn.metrics import (
    accuracy_score,
    roc_auc_score,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
    RocCurveDisplay,
    PrecisionRecallDisplay
)
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
# ============================
# 🔬 Experiment Tracking
# ============================
import mlflow
import mlflow.sklearn
from mlflow import MlflowClient

# ============================
# 🌐 Web / API / Networking
# ============================
import requests
from dotenv import load_dotenv

# ============================
# 🧪 Git & Version Control
# ============================
import git
from git import Repo, GitCommandError
import hashlib


# ============================
# 🧠 SHAP for Explainability
# ============================
import shap

# ============================
# 🧬 RDF & Provenance (rdflib)
# ============================
from rdflib import Graph, URIRef, Literal
from rdflib.namespace import PROV, XSD

# ============================
# ⚙️ System Monitoring
# ============================
import psutil


#Dataset metadata!!!

#Metadata from ZONEDO

In [14]:
import requests

def extract_dataset_metadata_from_doi(doi: str) -> dict:
    base_url = f"https://api.datacite.org/dois/{doi.lower()}"
    r = requests.get(base_url)
    r.raise_for_status()
    meta = r.json().get("data", {}).get("attributes", {})

    # Extract fields
    title = meta.get("titles", [{}])[0].get("title", "info not available")
    creators = [c.get("name", "") for c in meta.get("creators", [])]
    publisher = meta.get("publisher", "info not available")
    pub_year = meta.get("publicationYear", "info not available")
    url = meta.get("url", f"https://doi.org/{doi}")

    dataset_metadata = {
        "dataset_id": doi,
        "dataset_title": title,
        "dataset_description": meta.get("descriptions", "info not available"),
        "dataset_creator": ", ".join(creators) if creators else "info not available",
        "dataset_publisher": publisher,
        "dataset_publication_date": pub_year,
        "dataset_version": meta.get("version", "info not available"),
        "dataset_license": meta.get("rightsList", "info not available"),
        "dataset_keywords": "info not available",  # not always exposed
        "dataset_access_url": url,
        "dataset_documentation": url,
        "metadata_standard": meta.get("types", {}).get("resourceTypeGeneral", "info not available"),
        "related_resources": url,

        # PROV-O traceability fields
        "prov_entity": title,
        "prov_activity": "Ingestion and Publication",
        "prov_agent_dataset_creator": ", ".join(creators) if creators else "info not available",
        "prov_used": url,
        "prov_wasDerivedFrom": doi,
        "prov_wasAttributedTo": ", ".join(creators) if creators else "info not available",
        "prov_startedAtTime": pub_year,
        "prov_role_dataset_creator": "Original Data Author",
        "prov_role_database_creator": "Database Ingestor and Maintainer"
    }

    return dataset_metadata


In [15]:
# extract_dataset_metadata_from_doi("10.24432/C56C76") #dataset related metadata logging 


In [16]:
import requests

DB_API = "http://localhost/api/database/{db_id}"
HISTORY_API = "http://localhost/api/database/{db_id}/table/{table_id}/history"

def fetch_db_dataset_metadata(
    db_id: str,
    table_id: str,
    selected_version: str,
    target_variable: str,
    num_samples: int
) -> dict:
    try:
        # Fetch main DB metadata
        db_url = DB_API.format(db_id=db_id)
        db_response = requests.get(db_url)
        db_response.raise_for_status()
        db_data = db_response.json()
        print(db_data)

        # Fetch table history metadata
        history_url = HISTORY_API.format(db_id=db_id, table_id=table_id)
        history_response = requests.get(history_url)
        timestamp = "info not available"
        if history_response.status_code == 200:
            history_data = history_response.json()
            print(history_data)
            if isinstance(history_data, list) and len(history_data) > 0:
                timestamp = history_data[0].get("timestamp", timestamp)

        # Build flat metadata structure for DB storage
        dataset_metadata = {
            # Basic identity
            "dataset_id": table_id,
            "dataset_name": next(
                (t.get("name") for t in db_data.get("tables", []) if t.get("id") == table_id),
                "table name not available"
            ),
            "dataset_version": selected_version,
            "dataset_title": db_data.get("name", "info not available"),
            "dataset_description": db_data.get("description", "info not available"),

            # Ownership and access
            "dataset_creator": "info not available",
            "dataset_publisher": db_data.get("owner", {}).get("name", "info not available"),
            "dataset_access_url": db_url,
            "dataset_publication_date": timestamp,
            "dataset_license": "info not available",

            # Structure
            "columns": db_data.get("columns", "info not available"),
            "dataset_dataset_type": "tabular",
            "target_variable": target_variable,
            "ml_task": "classification",
            "num_samples": num_samples,

            # FAIR4ML placeholders
            "data_distribution": "info not available",
            "known_issues": "info not available",
            "trainedOn": "info not available",
            "testedOn": "info not available",
            "validatedOn": "info not available",
            "modelRisks": "info not available",
            "usageInstructions": "info not available",
            "ethicalLegalSocial": "info not available",

            # PROV-style fields
            "prov_entity": db_data.get("name", "info not available"),
            "prov_activity": "Ingestion and Publication",
            "prov_agent_dataset_creator": "info not available",
            "prov_agent_database_creator": db_data.get('owner', {}).get('name', 'info not available'),
            "prov_wasGeneratedBy": db_data.get('owner', {}).get('name', 'info not available'),
            "prov_used": db_url,
            "prov_wasDerivedFrom": "info not available",
            "prov_wasAttributedTo": "info not available",
            "prov_wasAssociatedWith": db_data.get('owner', {}).get('name', 'info not available'),
            "prov_startedAtTime": "info not available",
            "prov_endedAtTime": timestamp,
            "prov_location": db_url,
            "prov_role_dataset_creator": "",
            "prov_role_database_creator": "Database Ingestor and Maintainer"
        }

        return dataset_metadata

    except requests.exceptions.RequestException as e:
        print(f"[⚠️ Error] Failed to fetch DB metadata for {db_id}: {e}")
        return {}


Fetch info needed to fetch metadata:

In [17]:
# Mapping of version tags to table UUIDs
version_to_table_id = {
    "v0": "be7d1b8c-20fe-46f9-af52-488beaa9401a",  # Original
    "v1": "7e8e232d-15cd-425c-bf19-0f3c6dcaef98",  # Duplicated
    "v2": "f207790f-1b7d-436f-90ef-c15caa86e5dd",  # First 100
    "v3": "509e524a-8f2e-40e8-971c-3092fdbd1229",  # Shuffled
    "v4": "3f9dfdf0-b555-4ba4-89a2-92d28eaa9f5c" ,  # Normalized
    "v5": "741f0485-7428-4da6-b1de-08775f3f2f9b"   # Random
    

}

db_id = "1a867193-f56d-4180-92ae-ebdc63b68472"  # Static DB ID

def select_dataset_version():
    print("Select dataset version:")
    print("  v0 - Original")
    print("  v1 - Duplicated")
    print("  v2 - First 100")
    print("  v3 - Shuffled")
    print("  v4 - Normalized")
    print("  v5 - Random")
    
    selected_version = input("Enter version (v0–v5): ").strip().lower()
    
    if selected_version not in version_to_table_id:
        raise ValueError(f"❌ Invalid version selected: {selected_version}")
    
    selected_table_id = version_to_table_id[selected_version]
    
    print(f"\n✅ You selected version '{selected_version}' → Table ID: {selected_table_id}\n")
    
    return selected_version, selected_table_id

# Usage: #TODO CALL
selected_version, selected_table_id = select_dataset_version()


Select dataset version:
  v0 - Original
  v1 - Duplicated
  v2 - First 100
  v3 - Shuffled
  v4 - Normalized
  v5 - Random


Enter version (v0–v5):  v5



✅ You selected version 'v5' → Table ID: 741f0485-7428-4da6-b1de-08775f3f2f9b



In [18]:
import os
import json
import mlflow
# 
def log_metadata_dict_to_mlflow(metadata: dict, prefix: str = "", snapshot_name: str = "metadata_snapshot.json"):
    """
    Logs a flat metadata dictionary to MLflow:
    - Adds prefix to each key if provided (e.g., "session_")
    - Skips empty values
    - Logs a full JSON artifact for traceability
    """
    
    def safe_tag(key, value):
        if not mlflow.active_run():
            raise RuntimeError("❌ No active MLflow run.")
        
        key_clean = key.replace(":", "_").replace("/", "_").replace(" ", "_")
        try:
            val_str = json.dumps(value) if isinstance(value, (dict, list)) else str(value)
            if len(val_str) > 5000:
                val_str = val_str[:5000] + "...[TRUNCATED]"
            if len(key_clean) > 255:
                print(f"⚠️ Skipped tag (key too long): {key_clean}")
                return
            mlflow.set_tag(key_clean, val_str)
            print(f"✅ Logged tag: {key_clean}")
        except Exception as e:
            print(f"[⚠️ Error logging tag] {key_clean}: {e}")

    for key, value in metadata.items():
        if value not in [None, ""]:
            full_key = f"{prefix}{key}" if prefix else key
            safe_tag(full_key, value)

    # Save full metadata snapshot as JSON artifact
    os.makedirs("metadata", exist_ok=True)
    full_path = os.path.join("metadata", snapshot_name)
    with open(full_path, "w", encoding="utf-8") as f:
        json.dump(metadata, f, indent=2)
    
    mlflow.log_artifact(full_path, artifact_path="metadata")
    print(f"📁 Full metadata snapshot logged as: {snapshot_name}")


DBREPO INTEGRETION: API call to fetch the dataset for training

In [19]:
# API endpoint URL
API_URL = f"http://localhost/api/database/{db_id}/table/{selected_table_id}/data?size=100000&page=0"

# Define the headers
headers = {
    "Accept": "application/json"  # Specify the expected response format
}

try:
    # Send a GET request to the API with the Accept header
    response = requests.get(API_URL, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        dataset = response.json()
        
        
        print( dataset)
    else:
        print(f"Error: Received status code {response.status_code}")
        print("Response content:", response.text)
       

except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")


[{'id': '1', 'sepallengthcm': '6.700000000000000000', 'sepalwidthcm': '3.200000000000000000', 'petallengthcm': '5.600000000000000000', 'petalwidthcm': '1.600000000000000000', 'species': 'Iris-virginica'}, {'id': '2', 'sepallengthcm': '6.100000000000000000', 'sepalwidthcm': '2.800000000000000000', 'petallengthcm': '3.900000000000000000', 'petalwidthcm': '1.200000000000000000', 'species': 'Iris-versicolor'}, {'id': '3', 'sepallengthcm': '4.900000000000000000', 'sepalwidthcm': '3.700000000000000000', 'petallengthcm': '1.500000000000000000', 'petalwidthcm': '0.200000000000000000', 'species': 'Iris-setosa'}, {'id': '4', 'sepallengthcm': '6.200000000000000000', 'sepalwidthcm': '3.300000000000000000', 'petallengthcm': '4.600000000000000000', 'petalwidthcm': '1.500000000000000000', 'species': 'Iris-versicolor'}, {'id': '5', 'sepallengthcm': '6.600000000000000000', 'sepalwidthcm': '3.500000000000000000', 'petallengthcm': '5.600000000000000000', 'petalwidthcm': '2.000000000000000000', 'species':

replacing dynamic fetching of data When and if DBREPO isnt running (BACKUP)

In [20]:
# # 1. Read the JSON file id the API isnt available this data is saved locally but the data is from the API endpoint
# with open("iris_data.json", "r") as f:
#     dataset = json.load(f)


# ============================
# 📂 Setup MLflow
# ============================

In [21]:
import os
import mlflow

# Ensure tracking directory exists
project_dir = os.getcwd()
mlrunlogs_dir = os.path.join(project_dir, "mlrunlogs")
os.makedirs(mlrunlogs_dir, exist_ok=True)

# Set MLflow tracking URI (local SQLite backend)
mlflow_tracking_path = os.path.join(mlrunlogs_dir, "mlflow.db")
mlflow.set_tracking_uri("mlrunlogs/mlflow.db")

# Prompt for experiment name
experiment_name = input("Enter experiment name for MLflow: ").strip()
if not experiment_name:
    experiment_name = "default_experiment"
    print("⚠️ No name entered. Using fallback:", experiment_name)

mlflow.set_experiment(experiment_name)


Enter experiment name for MLflow:  jkl-


2025/06/04 15:03:04 INFO mlflow.tracking.fluent: Experiment with name 'jkl-' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///C:/Users/reema/REPO/notebooks/RQ_notebooks/mlrunlogs/mlflow.db/522819434955786530', creation_time=1749042184737, experiment_id='522819434955786530', last_update_time=1749042184737, lifecycle_stage='active', name='jkl-', tags={}>

# ============================
# 🔄 Git Commit Hash for previous commit for metadata
# ============================

In [22]:
import git
import os

def get_latest_git_commit(repo_path: str = "C:/Users/reema/REPO") -> dict:
    """
    Returns the latest Git commit metadata from the given repo path.
    """
    try:
        repo = git.Repo(repo_path)
        commit = repo.head.commit
        commit_metadata = {
            "git_commit": commit.hexsha,
            "git_author": commit.author.name,
            "git_email": commit.author.email,
            "git_commit_time": str(commit.committed_datetime),
            "git_message": commit.message.strip(),
            "git_branch": repo.active_branch.name if not repo.head.is_detached else "detached"
        }
        return commit_metadata

    except Exception as e:
        print(f"[⚠️ Git Error] Could not read Git repo at {repo_path}: {e}")
        return {
            "git_commit": "not available",
            "git_author": "not available",
            "git_email": "not available",
            "git_commit_time": "not available",
            "git_message": "not available",
            "git_branch": "not available"
        }

# Usage
repo_dir = "C:/Users/reema/REPO"
git_metadata = get_latest_git_commit(repo_dir)


# ============================
# Make threadpoolctl safe so MLflow’s autologger won’t crash ───
# ============================

In [23]:
# ─── Patch threadpoolctl if needed to avoid autolog crashes ───
try:
    import threadpoolctl
    _original_threadpool_info = threadpoolctl.threadpool_info

    def _safe_threadpool_info(*args, **kwargs):
        try:
            return _original_threadpool_info(*args, **kwargs)
        except Exception:
            return []

    threadpoolctl.threadpool_info = _safe_threadpool_info
except ImportError:
    pass  # If threadpoolctl isn't installed, we just skip this patch

# ─── Enable MLflow autologging (generic, works with sklearn and more) ───
import mlflow

mlflow.autolog(
    log_input_examples=True,
    log_model_signatures=True
)


2025/06/04 15:03:10 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/06/04 15:03:11 INFO mlflow.tracking.fluent: Autologging successfully enabled for statsmodels.


In [24]:
import hashlib
import json
import platform
import psutil
import numpy as np
import pandas as pd
from datetime import datetime
from subprocess import check_output, CalledProcessError

def log_standard_metadata(
    model_name: str,
    model,
    hyperparams: dict,
    acc: float,
    prec: float,
    rec: float,
    f1: float,
    auc: float,
    label_map: dict,
    run_id: str,
    test_size: float,
    random_state: int,
    id_cols: list,
    target_col: str,
    X,
    y,
    run_data=None
):
    # === Experiment Metadata ===
    mlflow.set_tag("run_id", run_id)  # [MLflow / DB anchor]
    mlflow.set_tag("model_name", model_name)  # [ML Metadata, FAIR]
    mlflow.set_tag("model_architecture", model.__class__.__name__)  # [MLSEA]
    mlflow.set_tag("test_size", test_size)  # [MLSEA, Reproducibility]
    mlflow.set_tag("random_state", random_state)  # [MLSEA, Reproducibility]

    # === Evaluation Metrics ===
    mlflow.set_tag("accuracy", acc)
    mlflow.set_tag("precision_macro", prec)
    mlflow.set_tag("recall_macro", rec)
    mlflow.set_tag("f1_macro", f1)
    mlflow.set_tag("roc_auc", auc)

    # === Hyperparameters and Label Encoding ===
    mlflow.set_tag("hyperparameters", json.dumps(hyperparams))  # [FAIR, MLSEA]
    mlflow.set_tag("label_map", json.dumps(label_map))  # [ML Preprocessing]

    # === Preprocessing Snapshot ===
    preprocessing_info = {
        "dropped_columns": id_cols,
        "numeric_columns": list(X.columns),
        "target_column": target_col,
        "stratified": False,
        "coercion_strategy": "Numeric cast (auto)",
        "feature_engineering": "None",
        "missing_value_strategy": "None",
        "outlier_detection": "None",
        "encoding_strategy": "LabelEncoder (target only)",
        "scaling": "None",
        "sampling": "None",
        "feature_selection": "None",
        "train_test_split": {"test_size": test_size, "random_state": random_state},
        "imbalance_ratio": str(dict(zip(*np.unique(y, return_counts=True)))),
        "preprocessing_timestamp": datetime.now().isoformat()
    }
    preprocessing_hash = hashlib.sha256(json.dumps(preprocessing_info).encode()).hexdigest()
    mlflow.set_tag("preprocessing_info", json.dumps(preprocessing_info))  # [MLSEA]
    mlflow.set_tag("preprocessing_hash", preprocessing_hash)

    # === Reproducibility ===
    mlflow.set_tag("model_serialization", "pickle")  # [FAIR, MLSEA]
    mlflow.set_tag("model_path", f"{model_name}.pkl")

    try:
        sha = check_output(["git", "rev-parse", "HEAD"], text=True).strip()
    except CalledProcessError:
        sha = "unknown"
    mlflow.set_tag("git_commit", sha)

    # === Compute Environment ===
    compute_env = {
        "os": f"{platform.system()} {platform.release()}",
        "cpu": platform.processor(),
        "ram_gb": round(psutil.virtual_memory().total / (1024 ** 3), 2),
        "python_version": platform.python_version(),
        "sklearn_version": sklearn.__version__,
        "pandas_version": pd.__version__,
        "numpy_version": np.__version__,
    }
    mlflow.set_tag("compute_environment", json.dumps(compute_env))  # [Reproducibility]

    # === Optional: Tag MLflow Justifications (previously logged manually) ===
    if run_data:
        for key, val in run_data.tags.items():
            if key.startswith("justification_"):
                mlflow.set_tag(key, val)


In [25]:
import os
import yaml
import numpy as np
from datetime import datetime

def generate_reproducibility_txt_log(
    model_name: str,
    dataset_name: str,
    dataset_version: str,
    hyperparams: dict,
    metrics: dict,
    git_commit: str,
    run_id: str,
    architecture_file_path: str = "provenance_architecture_description.txt"
) -> str:
    """
    Generate a reproducibility log (YAML + architecture) and return the saved path.
    This log combines:
    - Model and dataset details
    - Hyperparameters and evaluation metrics
    - Git provenance info
    - Reproduction steps
    - Provenance architecture description
    """

    def clean_values(d):
        """Convert numpy floats to native floats."""
        return {k: float(v) if isinstance(v, (np.float32, np.float64)) else v for k, v in d.items()}

    timestamp = datetime.utcnow().strftime("%Y-%m-%d %H:%M UTC")

    repro_data = {
        "📌 Model Details": {
            "Model Name": model_name,
            "Dataset Name": dataset_name,
            "Dataset Version": dataset_version,
            "Run ID": run_id,
            "Timestamp": timestamp
        },
        "🛠️ Hyperparameters": clean_values(hyperparams),
        "📈 Metrics": clean_values(metrics),
        "🔗 Git Info": {
            "Commit Hash": git_commit,
            "Reproduce With": f"git checkout {git_commit}"
        },
        "🚀 Reproduction Guide": [
            "1. Clone the repo and checkout the commit:",
            f"   git checkout {git_commit}",
            "2. Load and preprocess the dataset exactly as during training.",
            "3. Load the model using MLflow:",
            f"   mlflow.sklearn.load_model('runs:/{run_id}/model')",
            "4. Run inference or evaluation using the same pipeline/script."
        ]
    }

    # 🔐 Create and write to output file
    save_dir = os.path.join("MODEL_PROVENANCE", model_name)
    os.makedirs(save_dir, exist_ok=True)
    txt_path = os.path.join(save_dir, f"{model_name}_reproducibility.txt")

    with open(txt_path, "w", encoding="utf-8") as repro_file:
        yaml.dump(repro_data, repro_file, allow_unicode=True, sort_keys=False, width=100)
        repro_file.write("\n\n")

        if os.path.exists(architecture_file_path):
            with open(architecture_file_path, "r", encoding="utf-8") as arch_file:
                architecture_description = arch_file.read()
                repro_file.write(architecture_description)
        else:
            repro_file.write("[⚠️ Missing architecture description file]\n")

    return txt_path


In [26]:
def log_with_justification(log_func, key: str, value, context: str = ""):
    """
    Log a value using the specified MLflow log function (e.g., mlflow.log_param),
    then prompt the user for a justification and log it as a tag.
    """
    log_func(key, value)
    print(f"\n📝 Justification for `{key}` ({context})")
    user_reason = input("→ Why did you choose this value? ")
    mlflow.set_tag(f"justification_{key}", user_reason or "No justification provided")

def log_justification(key: str, question: str):
    """
    Prompt for a justification only (without logging a value), and log it as a tag.
    """
    print(f"\n📝 Justification for `{key}`")
    user_reason = input(f"→ {question} ")
    mlflow.set_tag(f"justification_{key}", user_reason or "No justification provided")


In [27]:
# mlflow.end_run()

# ============================
# 🚀 Start MLflow Run 
# ============================

In [ ]:
import hashlib
from datetime import datetime
import mlflow
import pandas as pd

with mlflow.start_run() as run:
    client = MlflowClient()
    run_data = client.get_run(run.info.run_id).data
    # ─────────────── Session Metadata ─────────────────────────────────────
    session_metadata = collect_session_metadata(prompt_fields=True)
    mlflow.log_params(session_metadata)  # [PROV, Internal] Session and environment context

    # ─────────────── Dataset Metadata ─────────────────────────────────────
    doi_metadata = extract_dataset_metadata_from_doi("10.24432/C56C76")  # [FAIR, PROV, FAIR4ML]

    # ─────────────── Experiment Start Time ────────────────────────────────
    start_time = datetime.now().isoformat()
    mlflow.set_tag("startedAtTime", start_time)  # [PROV] Activity start time

    #######################################################################
    ### Preprocessing #####################################################

    # ── Load into a DataFrame ────────────────────────────────────────────
    df = pd.DataFrame(dataset)
    original_row_count = df.shape[0]
    mlflow.log_param("input_row_count", original_row_count)  # [MLSEA] Input data size

    # Log column names before transformation
    mlflow.set_tag("raw_columns", ','.join(df.columns))  # [FAIR4ML, Internal]

    # ── Generate row hashes ───────────────────────────────────────────────
    before_hashes = set(df.astype(str).apply(lambda row: hash(tuple(row)), axis=1))
    mlflow.set_tag("row_hash_tracking", "enabled")  # [Internal] Used for provenance/repeatability

    # ── Extract target variable ───────────────────────────────────────────
    target_col = df.columns[-1]
    mlflow.set_tag("target_variable", target_col)  # [FAIR4ML, MLSEA]

    # ── Separate features and labels ──────────────────────────────────────
    y = df[target_col]
    X = df.drop(columns=[target_col])
    mlflow.set_tag("feature_columns", ','.join(X.columns))  # [FAIR4ML, MLSEA]

    # ── Drop ID columns (case-insensitive) ────────────────────────────────
    id_cols = [c for c in X.columns if c.lower() == "id"]
    if id_cols:
        X = X.drop(columns=id_cols)
        mlflow.set_tag("dropped_id_columns", ','.join(id_cols))  # [Internal]

    # ── Convert columns to numeric where possible ─────────────────────────
    numeric_conversion_count = 0
    for c in X.columns:
        try:
            X[c] = pd.to_numeric(X[c])
            numeric_conversion_count += 1
        except Exception:
            continue
    mlflow.log_param("numeric_columns_converted", numeric_conversion_count)  # [Internal, FAIR4ML]

    # ── Print diagnostic info ─────────────────────────────────────────────
    print("ML_EXP_Shapes:", X.shape, y.shape)
    mlflow.log_param("feature_matrix_shape", str(X.shape))  # [MLSEA]
    mlflow.log_param("label_vector_shape", str(y.shape))    # [MLSEA]
#######################################################################################################
### 8) Label Encoding and Metadata Logging ############################################################

# ── Encode class labels numerically ────────────────────────────────────────────────────────
    le = LabelEncoder()
    y = le.fit_transform(y)
    print("ML_EXP_Classes:", le.classes_)
    
    mlflow.set_tag("class_names", ','.join(le.classes_))  # [FAIR4ML, MLSEA]
    
    # ── Count rows and hash comparison before vs after preprocessing ────────────────────────────
    count_end = df.shape[0]
    after_hashes = set(df.astype(str).apply(lambda row: hash(tuple(row)), axis=1))
    
    n_insert = len(after_hashes - before_hashes)
    n_delete = len(before_hashes - after_hashes)
    
    #######################################################################################################
    ### Metadata Logging (Standardized Format) ############################################################
    
    # ── Extended DB Metadata ────────────────────────────────────────────────────────────────────
    db_meta = fetch_db_dataset_metadata(db_id, selected_table_id, selected_version, target_col, df.shape[0])  # [Internal]
    
    mlflow.set_tag("Internal_DBRepo_table_last_modified", db_meta.get("dataset_publication_date", "unknown"))
  # [PROV]
    
    # ── Row Count Metrics ────────────────────────────────────────────────────────────────────────
    mlflow.log_metric("row_count_start", original_row_count)              # [MLSEA, FAIR4ML]
    mlflow.log_metric("row_count_end", count_end)                  # [MLSEA, FAIR4ML]
    mlflow.log_metric("num_inserted_rows", n_insert)               # [PROV]
    mlflow.log_metric("num_deleted_rows", n_delete)                # [PROV]
    
    # ── Raw Data Source Metadata ─────────────────────────────────────────────────────────────────
    mlflow.set_tag("data_source", API_URL)                         # [FAIR]
    mlflow.log_param("retrieval_time_utc", datetime.utcnow().isoformat())  # [PROV]
    mlflow.log_param("raw_row_count", len(df))                     # [MLSEA]
    mlflow.log_param("raw_columns", df.columns.tolist())           # [FAIR4ML]
    mlflow.log_param("dropped_columns", id_cols)                   # [Internal]
    
    # ── Post-Processing Metadata ─────────────────────────────────────────────────────────────────
    mlflow.log_param("final_num_features", X.shape[1])             # [MLSEA]
    mlflow.log_param("final_feature_names", X.columns.tolist())    # [FAIR4ML]
    mlflow.set_tag("target_variable_encoded", target_col)          # [FAIR4ML]
    
    # ── Label Mapping as Artifact ───────────────────────────────────────────────────────────────
    label_map = {int(idx): cls for idx, cls in enumerate(le.classes_)}
    buffer = io.StringIO()
    json.dump(label_map, buffer, indent=2)
    buffer.seek(0)
    mlflow.log_text(buffer.getvalue(), artifact_file="label_mapping.json")  # [FAIR4ML]
    
    # ── Training Metadata ────────────────────────────────────────────────────────────────────────
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    model_name = f"RandomForest_Iris_v{ts}"
    mlflow.set_tag("model_name", model_name)                       # [MLSEA]
    
    train_start_ts = datetime.now().isoformat()
    mlflow.set_tag("training_start_time", train_start_ts)          # [PROV]
########################################################################################################
### Model Parameters & Split Metadata ##################################################################

# ── Prompt test size and seed ─────────────────────────────────────────────────────────────────────────
    try:
        test_size = float(input("Enter test size (e.g., 0.2 for 20% test set): "))
    except ValueError:
        print("Invalid input. Defaulting to 0.2")
        test_size = 0.2
    
    try:
        random_state = int(input("Enter random seed (e.g., 42): "))
    except ValueError:
        print("Invalid input. Defaulting to 42")
        random_state = 42
    
    # ── Train/test split ────────────────────────────────────────────────────────────────────────────────
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    # ── Log split config ────────────────────────────────────────────────────────────────────────────────
    mlflow.log_param("test_size", test_size)                     # [MLSEA]
    mlflow.log_param("random_seed", random_state)               # [PROV]
    mlflow.log_param("n_train_samples", X_train.shape[0])       # [FAIR4ML]
    mlflow.log_param("n_test_samples",  X_test.shape[0])        # [FAIR4ML]
    mlflow.log_param("n_features",      X_train.shape[1])       # [MLSEA]
    
    ########################################################################################################
    ### Model Selection & Hyperparameters ##################################################################
    
    # ── Define hyperparameters ───────────────────────────────────────────────────────────────────────────
    ML_EXP_hyperparams = {
        "n_estimators":       100,
        "criterion":          "entropy",
        "max_depth":          10,
        "min_samples_split":  3,
        "min_samples_leaf":   1,
        "max_features":       "sqrt",
        "bootstrap":          True,
        "oob_score":          True,
        "class_weight":       None,
        "verbose":            1,
        "n_jobs":             -1
    }
    
    # ── Model selection ───────────────────────────────────────────────────────────────────────────────────
    available_models = {
        "random_forest": RandomForestClassifier,
        "decision_tree": DecisionTreeClassifier,
        "logistic_regression": LogisticRegression,
        "knn": KNeighborsClassifier,
        "svm": SVC,
        "gradient_boosting": GradientBoostingClassifier
    }
    
    # User prompt
    print("Choose a model to train:")
    for i, name in enumerate(available_models.keys()):
        print(f"{i + 1}. {name}")
    
    choice = input("Enter model number (default 1 for random_forest): ").strip()
    choice = int(choice) if choice else 1
    selected_key = list(available_models.keys())[choice - 1]
    selected_model_class = available_models[selected_key]
    mlflow.set_tag("selected_model", selected_key)  # [FAIR4ML, MLSEA]
    
    # ── Initialize model ────────────────────────────────────────────────────────────────────────────────
    model = selected_model_class(**ML_EXP_hyperparams)
    
    # ── Log hyperparameters with justification ───────────────────────────────────────────────────────────
    for key, val in ML_EXP_hyperparams.items():
        log_with_justification(mlflow.log_param, key, val, context="Hyperparameter configuration")  # [FAIR4ML, MLSEA]
    
    ########################################################################################################
    ### Model Training & Evaluation ########################################################################
    
    # ── Fit the model ────────────────────────────────────────────────────────────────────────────────────
    model.fit(X_train, y_train)
    train_end_ts = datetime.now().isoformat()
    mlflow.set_tag("training_end_time", train_end_ts)  # [PROV]
    
    # ── Predictions ──────────────────────────────────────────────────────────────────────────────────────
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)
    
    # ── Compute and log metrics ─────────────────────────────────────────────────────────────────────────
    acc  = accuracy_score(y_test, y_pred)
    auc  = roc_auc_score(y_test, y_proba, multi_class="ovr")
    prec = precision_score(y_test, y_pred, average="macro")
    rec  = recall_score(y_test,  y_pred, average="macro")
    f1   = f1_score(y_test,      y_pred, average="macro")
    
    mlflow.log_metric("accuracy", acc)              # [MLSEA]
    mlflow.log_metric("roc_auc", auc)               # [MLSEA]
    mlflow.log_metric("precision_macro", prec)      # [MLSEA]
    mlflow.log_metric("recall_macro", rec)          # [MLSEA]
    mlflow.log_metric("f1_macro", f1)               # [MLSEA]


########################################################################################################
### Final Logging: Justifications, Metrics, Environment, Dataset Metadata #############################

# ── Prompt for and log justifications ────────────────────────────────────────────────────────────────
    log_justification("model_choice", "Why did you choose this model (e.g., RandomForestClassifier) for this task?")
    log_justification("target_variable", "Why did you choose this column as the prediction target?")
    log_justification("test_split", "Why this train/test ratio (e.g., 80/20)?")
    log_justification("metric_choice", "Why did you use accuracy/f1/ROC-AUC as your evaluation metric?")
    log_justification("threshold_accuracy", "Was there a threshold for accuracy? Why?")
    log_justification("dataset_version", "Why did you use this specific dataset version?")
    log_justification("drop_column_X", "Why did you drop any specific columns from the dataset?")
    log_justification("experiment_name", "Any context behind this experiment name or setup?")
    log_justification("model_limitations", "Any known model limitations?")
    log_justification("ethical_considerations", "Any known model ethical considerations?")
    log_justification("intended_use", "Known model intended use?")
    log_justification("not_intended_for", "Model not_intended_for?")


    # ── Log model evaluation metrics ────────────────────────────────────────────────────────────────────
    mlflow.log_metric("precision_macro", prec)    # [MLSEA]
    mlflow.log_metric("recall_macro", rec)        # [MLSEA]
    mlflow.log_metric("f1_macro", f1)             # [MLSEA]
    mlflow.log_metric("accuracy", acc)            # [MLSEA]
    mlflow.log_metric("roc_auc", auc)             # [MLSEA]
    
    # ── Log environment info ─────────────────────────────────────────────────────────────────────────────
    mlflow.log_params({
        "python_version":       platform.python_version(),
        "os_platform":          f"{platform.system()} {platform.release()}",
        "sklearn_version":      sklearn.__version__,
        "pandas_version":       pd.__version__,
        "numpy_version":        np.__version__,
        "matplotlib_version":   matplotlib.__version__,
        "seaborn_version":      sns.__version__,
        "shap_version":         shap.__version__,
    })  # [PROV, Internal]
    
    # ── Tag notebook name ────────────────────────────────────────────────────────────────────────────────
    mlflow.set_tag("notebook_name", "RQ1_2.ipynb")  # [Internal]
    
    # ── Dataset metadata tags ────────────────────────────────────────────────────────────────────────────
    mlflow.set_tag("dataset_name",    db_meta.get("dataset_name", "unknown") )    # [FAIR4ML, PROV]
    mlflow.set_tag("dataset_version", selected_version)                                           # [FAIR4ML, Internal]
    mlflow.set_tag("dataset_id",      selected_table_id)  # [FAIR4ML, Internal]

########################################################################################################
### Plots: Feature Importance, ROC, PR, Confusion Matrix, SHAP #########################################

# ── Create plot output directory ─────────────────────────────────────────────────────────────────────
    # plot_dir = os.path.join("ML_EXP_plots", run.info.run_id) ##TODO test this path change

    summary_dir = os.path.join(os.getcwd(), "MODEL_PROVENANCE", model_name)
    os.makedirs(summary_dir, exist_ok=True)
    
    # os.makedirs(plot_dir, exist_ok=True)
    plot_dir = summary_dir  # 👈 Use the same directory as the summary

    # ── 1) Feature Importance Bar Chart ──────────────────────────────────────────────────────────────────
    if hasattr(model, "feature_importances_"):
        importances = model.feature_importances_
        feature_names = getattr(X_train, "columns", [f"f{i}" for i in range(X_train.shape[1])])
        
        fi_path = os.path.join(plot_dir, "feature_importances.png")
        plt.figure(figsize=(8, 6))
        sns.barplot(x=importances, y=feature_names)
        plt.title("Feature Importances")
        plt.xlabel("Importance")
        plt.ylabel("Feature")
        plt.tight_layout()
        plt.savefig(fi_path)
        mlflow.log_artifact(fi_path)  # [MLSEA]
        plt.close()
    
    # ── 2) Multi-class ROC Curves ───────────────────────────────────────────────────────────────────────
    classes = np.unique(y_test)
    y_test_bin = label_binarize(y_test, classes=classes)
    
    for idx, cls in enumerate(classes):
        disp = RocCurveDisplay.from_predictions(y_test_bin[:, idx], y_proba[:, idx], name=f"ROC for class {cls}")
        roc_path = os.path.join(plot_dir, f"roc_curve_cls_{cls}.png")
        disp.figure_.savefig(roc_path)
        mlflow.log_artifact(roc_path)  # [MLSEA]
        plt.close(disp.figure_)
    
    # ── 3) Multi-class Precision-Recall Curves ───────────────────────────────────────────────────────────
    for idx, cls in enumerate(classes):
        disp = PrecisionRecallDisplay.from_predictions(y_test_bin[:, idx], y_proba[:, idx], name=f"PR curve for class {cls}")
        pr_path = os.path.join(plot_dir, f"pr_curve_cls_{cls}.png")
        disp.figure_.savefig(pr_path)
        mlflow.log_artifact(pr_path)  # [MLSEA]
        plt.close(disp.figure_)
    
    # ── 4) Confusion Matrix Plot ─────────────────────────────────────────────────────────────────────────
    cm_path = os.path.join(plot_dir, "confusion_matrix.png")
    cm = confusion_matrix(y_test, y_pred)
    
    plt.figure(figsize=(6, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.tight_layout()
    plt.savefig(cm_path)
    mlflow.log_artifact(cm_path)  # [MLSEA]
    plt.close()
    
    # ── 5) SHAP Summary Plot ─────────────────────────────────────────────────────────────────────────────
    shap_path = os.path.join(plot_dir, "shap_summary.png")
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_test)
    
    shap.summary_plot(shap_values, X_test, show=False)
    plt.tight_layout()
    plt.savefig(shap_path)
    mlflow.log_artifact(shap_path)  # [FAIR4ML, MLSEA]
    plt.close()
    
    ########################################################################################################
    ### Final: Metadata Summary Logging ####################################################################
    
    
    
    # log_standard_metadata(
    #     model_name=model_name,
    #     model=model,
    #     hyperparams=ML_EXP_hyperparams,
    #     acc=acc,
    #     prec=prec,
    #     rec=rec,
    #     f1=f1,
    #     auc=auc,
    #     label_map=label_map,
    #     run_id=run.info.run_id,
    #     test_size=test_size,
    #     random_state=random_state,
    #     run_data=run_data
    # )
    log_standard_metadata(
    model_name=model_name,
    model=model,
    hyperparams=ML_EXP_hyperparams,
    acc=acc,
    prec=prec,
    rec=rec,
    f1=f1,
    auc=auc,
    label_map=label_map,
    run_id=run.info.run_id,
    test_size=test_size,
    random_state=random_state,
    id_cols=id_cols,         # ✅ list of dropped ID columns
    target_col=target_col,   # ✅ your target column, likely defined as df.columns[-1]
    X=X,                     # ✅ your features DataFrame
    y=y,                     # ✅ your labels array or Series
    run_data=run_data        # optional but useful
    )

########################################################################################################
### Export Model (.pkl) and Log as Artifact ############################################################

# ── Define output path ───────────────────────────────────────────────────────────────────────────────
    pkl_path = f"Trained_models/{model_name}.pkl"
    os.makedirs("Trained_models", exist_ok=True)  # Ensure the folder exists
    
    # ── Serialize the trained model to disk ──────────────────────────────────────────────────────────────
    with open(pkl_path, "wb") as f:
        pickle.dump(model, f)
    
    # ── Log the serialized model to MLflow as an artifact ────────────────────────────────────────────────
    mlflow.log_artifact(pkl_path, artifact_path=model_name)  # [FAIR4ML, MLSEA]

########################################################################################################
### COMMIT: Git Integration + Provenance Logging #######################################################

    def get_latest_commit_hash(repo_path="."):
        res = subprocess.run(
            ["git", "-C", repo_path, "rev-parse", "HEAD"],
            capture_output=True, text=True, check=True
        )
        return res.stdout.strip()
    
    def get_remote_url(repo_path=".", remote="origin"):
        res = subprocess.run(
            ["git", "-C", repo_path, "config", "--get", f"remote.{remote}.url"],
            capture_output=True, text=True, check=True
        )
        return res.stdout.strip()
    
    def make_commit_link(remote_url, commit_hash):
        base = remote_url.rstrip(".git")
        if base.startswith("git@"):
            base = base.replace(":", "/").replace("git@", "https://")
        return f"{base}/commit/{commit_hash}"
    
    def simple_commit_and_push_and_log(repo_path=".", message="Auto commit", remote="origin", branch="main"):
        status = subprocess.run(["git", "-C", repo_path, "status", "--porcelain"], capture_output=True, text=True)
        if not status.stdout.strip():
            print("🟡 No changes to commit.")
            return None, None
    
        subprocess.run(["git", "-C", repo_path, "add", "--all"], capture_output=True, text=True)
        commit = subprocess.run(["git", "-C", repo_path, "commit", "-m", message], capture_output=True, text=True)
        if commit.returncode:
            print("❌ git commit failed:\n", commit.stderr)
            return None, None
        print("✅ Commit successful.")
    
        push = subprocess.run(["git", "-C", repo_path, "push", "-u", remote, branch], capture_output=True, text=True)
        if push.returncode:
            print("❌ git push failed:\n", push.stderr)
        else:
            print("🚀 Push successful.")
    
        sha = get_latest_commit_hash(repo_path)
        url = get_remote_url(repo_path, remote)
        link = make_commit_link(url, sha)
        return sha, link
    
    # ── Perform commit and get commit SHA and link ───────────────────────────────────────────────────────
    sha, link = simple_commit_and_push_and_log(
        repo_path=".",
        message="Auto commit after successful training"
    )
    
    # ── Ask for version tag and log it ───────────────────────────────────────────────────────────────────
    def get_version_tag_for_commit(commit_hash, known_tags=None):
        known_tags = known_tags or {}
        version_tag = known_tags.get(commit_hash, "untagged")
        if version_tag == "untagged":
            print(f"⚠️ Commit {commit_hash[:8]} is not tagged with a version.")
            user_input = input("🔖 Enter version tag for this commit (or press Enter to skip): ").strip()
            version_tag = user_input if user_input else "untagged"
        return commit_hash, version_tag
    
    commit, version_tag = get_version_tag_for_commit(sha)
    mlflow.set_tag("GIT_code_version", version_tag)  # [PROV]
    mlflow.set_tag("model_version", version_tag)  # [PROV]

    
    
    # ── Log author info ──────────────────────────────────────────────────────────────────────────────────
    def get_git_author():
        name = subprocess.check_output(["git", "config", "user.name"]).decode().strip()
        email = subprocess.check_output(["git", "config", "user.email"]).decode().strip()
        return name, email
    
    name, email = get_git_author()
    mlflow.set_tag("GIT_user", name)               # [PROV]
    mlflow.set_tag("GIT_user_email", email)        # [PROV]
    
    # ── Log Git diff between this and previous commit ────────────────────────────────────────────────────
    if sha and link:
        previous_commit_hash = db_meta.get("code_commit_hash", "")  # Fallback for comparison
        if previous_commit_hash:
            diff_text = subprocess.check_output(
                ["git", "-C", ".", "diff", previous_commit_hash, sha],
                encoding="utf-8", errors="ignore"
            )
    
            remote_url = get_remote_url(".")
            remote_url = remote_url.rstrip(".git")
            if remote_url.startswith("git@"):
                remote_url = remote_url.replace(":", "/").replace("git@", "https://")
    
            previous_commit_url = f"{remote_url}/commit/{previous_commit_hash}"
            current_commit_url  = f"{remote_url}/commit/{sha}"
    
            diff_data = {
                "GIT_previous_commit":        previous_commit_hash,
                "GIT_previous_commit_url":    previous_commit_url,
                "GIT_current_commit":         sha,
                "GIT_current_commit_url":     current_commit_url,
                "GIT_diff":                   diff_text
            }
    
            mlflow.log_dict(diff_data, artifact_file="GIT_commit_diff.json")  # [PROV]
            mlflow.set_tag("GIT_previous_commit_hash", previous_commit_hash)
            mlflow.set_tag("GIT_current_commit_hash", sha)
            mlflow.set_tag("GIT_current_commit_url", link)
########################################################################################################
### Reproducibility Metadata Extraction + Text Log #####################################################

# ── Log all categorized metadata (FAIR, PROV, DBRepo, etc.) ───────────────────────────────────────────
    # log_metadata_dict_to_mlflow(categorized_fields)  # [FAIR4ML, PROV, Internal]

    log_metadata_dict_to_mlflow(
        metadata=doi_metadata,
        prefix="DOI_",
        snapshot_name="doi_metadata_snapshot.json"
    )
    # ── Retrieve full run metadata ───────────────────────────────────────────────────────────────────────
    run_id    = run.info.run_id
    run_info  = client.get_run(run_id).info
    run_data  = client.get_run(run_id).data
    
    params  = dict(run_data.params)
    metrics = dict(run_data.metrics)
    tags    = dict(run_data.tags)
    
    # ── List all artifacts in the run ────────────────────────────────────────────────────────────────────
    artifact_uri  = run_info.artifact_uri
    artifact_meta = []
    
    def _gather(path=""):
        for af in client.list_artifacts(run_id, path):
            if af.is_dir:
                _gather(af.path)
            else:
                rel_path = af.path.lower()
                if rel_path.endswith((".json", ".txt", ".patch")):
                    artifact_meta.append({"path": af.path, "type": "text"})
                elif rel_path.endswith((".png", ".jpg", ".jpeg", ".svg")):
                    artifact_meta.append({"path": af.path, "type": "image"})
                else:
                    artifact_meta.append({"path": af.path, "type": "other"})
    
    _gather()
    
    # ── (Optional) Store artifact meta if needed ─────────────────────────────────────────────────────────
    mlflow.log_dict({"artifacts": artifact_meta}, artifact_file="artifact_summary.json")  # [Internal]
    
    # ── Notebook directory (for trace/log location reference) ────────────────────────────────────────────
    notebook_dir = os.getcwd()
    
    ########################################################################################################
    ### Generate Reproducibility Instructions ##############################################################
    
    # ── Generate reproducibility .txt log with key details ───────────────────────────────────────────────
    repro_txt_path = generate_reproducibility_txt_log(
        model_name=model_name,
        dataset_name=db_meta.get("dataset_name", "unknown"),
        dataset_version=selected_version,
        hyperparams=ML_EXP_hyperparams,
        metrics={
            "accuracy": acc,
            "f1_macro": f1,
            "precision_macro": prec,
            "recall_macro": rec,
            "roc_auc": auc
        },
        git_commit=sha,
        run_id=run_id
    )
    
    # ── Log the .txt path to MLflow for traceability ─────────────────────────────────────────────────────
    mlflow.log_param("reproducibility_log_path", repro_txt_path)  # [Internal, FAIR4ML]
########################################################################################################
### COMBINE: Export Full Run Summary as JSON ###########################################################

# ── Create output directory ──────────────────────────────────────────────────────────────────────────
    # summary_dir = os.path.join(os.getcwd(), "MODEL_PROVENANCE", model_name)
    # os.makedirs(summary_dir, exist_ok=True)
    
    # ── Prepare run summary dict ─────────────────────────────────────────────────────────────────────────
    summary = {
        "run_id":         run_id,
        "run_name":       run_info.run_name,
        "experiment_id":  run_info.experiment_id,
        "start_time":     run_info.start_time,
        "end_time":       run_info.end_time,
        "params":         params,
        "metrics":        metrics,
        "tags":           tags,
        "artifacts":      artifact_meta
    }
    
    # ── Write summary to JSON file ───────────────────────────────────────────────────────────────────────
    summary_filename    = f"{model_name}_run_summary.json"
    summary_local_path  = os.path.join(summary_dir, summary_filename)
    
    with open(summary_local_path, "w", encoding="utf-8") as f:
        json.dump(summary, f, indent=2)
    
    # ── Log summary JSON to MLflow ───────────────────────────────────────────────────────────────────────
    mlflow.log_artifact(summary_local_path, artifact_path="run_summaries")  # [FAIR4ML, Internal]
    print("📁 Run summary JSON logged at:", summary_local_path)
    
    # ── End MLflow run with PROV-O end timestamp ─────────────────────────────────────────────────────────
    end_time = datetime.now().isoformat()
    mlflow.set_tag("endedAtTime", end_time)  # [PROV]
    mlflow.end_run()



Enter your role (default: collaborator):  
Enter project ID (default: default_project):  



📌 Session Metadata:
  session_id: 4f957107-7a08-4242-beaa-f00e330a16b2
  username: reema
  timestamp_utc: 2025-06-04T13:03:14.485696
  hostname: Purplish
  platform: Windows
  os_version: 10.0.26100
  python_version: 3.11.5
  role: collaborator
  project_id: default_project
ML_EXP_Shapes: (150, 4) (150,)
ML_EXP_Classes: ['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']
{'id': '1a867193-f56d-4180-92ae-ebdc63b68472', 'name': 'Iris_Database', 'description': None, 'tables': [{'id': 'f207790f-1b7d-436f-90ef-c15caa86e5dd', 'name': 'Iris_Table_First100', 'alias': None, 'identifiers': [], 'owner': {'id': '0dc0f6c4-d584-103f-83db-6d7a64d2e2ec', 'username': 'reema26', 'name': 'Reema George Dass', 'orcid': None, 'qualified_name': 'Reema George Dass — @reema26', 'given_name': 'Reema George', 'family_name': 'Dass'}, 'description': 'Iris_Table_First100', 'columns': [{'id': 'cae5f08f-74fe-44c1-9d3a-d2dd34a8be2c', 'name': 'Id', 'alias': None, 'size': None, 'd': None, 'mean': 51, 'median': 51, 'conce

Enter test size (e.g., 0.2 for 20% test set):  


Invalid input. Defaulting to 0.2


Enter random seed (e.g., 42):  


...


📝 Justification for `verbose` (Hyperparameter configuration)


→ Why did you choose this value?  



📝 Justification for `n_jobs` (Hyperparameter configuration)


→ Why did you choose this value?  


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0

In [76]:
summary_dir

'C:\\Users\\reema\\REPO\\notebooks\\RQ_notebooks\\MODEL_PROVENANCE\\RandomForest_Iris_v20250519_122304'

In [34]:

required_keywords = [
    "mlflow", "scikit-learn", "pandas", "numpy", "pyyaml", "seaborn",
    "matplotlib", "shap", "rdflib", "requests", "python-dotenv", "gitpython", "psutil", "pyld"
]

# Run pip freeze
result = subprocess.run(["pip", "freeze"], stdout=subprocess.PIPE, text=True)
all_packages = result.stdout.splitlines()

# Filter based on matching names
filtered = [pkg for pkg in all_packages if any(kw.lower() in pkg.lower() for kw in required_keywords)]

# Save filtered requirements to file
filtered_requirements_path = "requirements.txt"
with open(filtered_requirements_path, "w") as f:
    f.write("\n".join(filtered))

filtered_requirements_path


'requirements.txt'

Table creations

TABLE LOGGING:

In [63]:
import json
import requests
from datetime import datetime

# --- Configuration ---
headers = {"Content-Type": "application/json"}
auth = ("reema1", "Toothless!26")
BASE = "http://localhost/api/database/633f5987-d116-42e8-97fc-36b9c25ade24/table"
TABLES = {
    "session_metadata": "0eec52d1-3cd0-4c5f-ab61-a7387dbe1449",
    "experiment_metadata": "35d83c73-9a63-4f95-950a-4af699f8c16a",
    "git_metadata": "09832028-c343-4bb1-82c0-440a78e78c78",
    "dataset_metadata": "79167d03-9afa-45dd-98aa-46647c021f9f",
    "model_metadata": "9d49d172-df6f-4aa2-9141-cb18865138f6",
    "metrics_justification": "e553f49e-1074-4642-b27c-39df9464c777"
}

# --- Load metadata file ---
# with open("MODEL_PROVENANCE/b788db5d12174c28bc175589898f7f95/RandomForest_Iris_v20250516_193049_run_summary.json", "r") as f:
with open(summary_local_path, "r") as f:

    meta = json.load(f)

# --- Helper ---
def to_mysql_datetime(ts):
    return datetime.strptime(ts.split(".")[0], "%Y-%m-%dT%H:%M:%S").isoformat() + "+00:00"

# --- Extract shared values ---
run_id = meta["run_id"]
session_id = meta["params"]["session_id"]
dataset_id = meta["tags"]["dataset_id"]
model_id = "model_" + meta["tags"]["model_name"].split("_")[1].lower()
git_commit = meta["tags"]["git_commit"]
git_version = meta["tags"]["GIT_code_version"]
timestamp_utc = meta["params"]["timestamp_utc"]
username = meta["params"]["username"]
platform = meta["params"]["platform"]
hostname = meta["params"]["hostname"]
target_var = meta["tags"]["target_variable"]
label_map = meta["tags"]["label_map"]
feature_list = meta["params"]["final_feature_names"]
dataset_name = meta["tags"]["dataset_name"]
dataset_version = meta["tags"]["dataset_version"]
estimator = meta["tags"]["estimator_name"]
feature_select = meta["tags"]["feature_columns"]
label_snap = meta["tags"]["target_variable_encoded"]
model_name = meta["tags"]["model_name"]
imbalance_ratio = 1.0 if "imbalance_ratio" not in meta["tags"] else meta["tags"]["imbalance_ratio"]

# --- Extract shared values (exact keys from JSON) ---
python_version = meta["params"].get("python_version")
os_platform = meta["params"].get("os_platform")
role = meta["params"].get("role")
project_id = meta["params"].get("project_id")
script_name = meta["params"].get("source_file_name", "None_specified")

# --- Build full session payload (VARCHAR timestamp, full mapping) ---
session_payload = {
    "session_id": session_id,
    "username": username,
    "timestamp": timestamp_utc,
    "hostname": hostname,
    "platform": platform,
    "python_version": python_version,
    "os_platform": os_platform,
    "role": role,
    "project_id": project_id,
    "script_name": script_name
}

session_url = f"{BASE}/{TABLES['session_metadata']}/data"

response = requests.post(
    session_url,
    headers=headers,
    auth=auth,
    json={"data": session_payload}
)

# --- Logging ---
print("\n🔍 Session Metadata POST")
print("➡️ URL:", session_url)
print("📦 Payload:")
print(json.dumps(session_payload, indent=2))
print("📤 Status Code:", response.status_code)
print("📝 Response Text:", response.text)
#######################################################

# # --- 2. Experiment Metadata ---
exp_payload = {
    "runid": run_id,
    "sessionid": session_id,
    "modelid": model_id,
    "datasetid": dataset_id,
    "git_commit": meta["tags"].get("git_commit","None_specified"),
    "invenioid": meta["tags"].get("DOI_dataset_id","None_specified"),
    "timestamp": timestamp_utc,  # keep as VARCHAR

    # NEW fields
    "experiment_id": meta["tags"].get("experiment_id","None_specified"),
    "run_name": meta.get("run_name","None_specified"),
    "training_start_time": meta["tags"].get("training_start_time","None_specified"),
    "training_end_time": meta["tags"].get("training_end_time","None_specified"),
    "source_file": meta["tags"].get("mlflow.source.name","None_specified"),
    "source_notebook": meta["tags"].get("notebook_name","None_specified")
}

exp_url = f"{BASE}/{TABLES['experiment_metadata']}/data"

response = requests.post(
    exp_url,
    headers=headers,
    auth=auth,
    json={"data": exp_payload}
)

# --- Logging ---
print("\n🔍 Experiment Metadata POST")
print("➡️ URL:", exp_url)
print("📦 Payload:")
print(json.dumps(exp_payload, indent=2))
print("📤 Status Code:", response.status_code)
print("📝 Response Text:", response.text)


# --- 3. Git Metadata ---
git_payload = {
    "commit_hash": meta["tags"]["git_commit"],
    "repo_url": meta["tags"].get("DOI_prov_used","None_specified"),
    "branch": meta["tags"].get("GIT_branch", "main"),
    "author": meta["tags"].get("GIT_user","None_specified"),
    "author_email": meta["tags"].get("GIT_user_email","None_specified"),
    "version": meta["tags"].get("GIT_code_version","None_specified"),
    "origin_url": meta["tags"].get("GIT_origin_url","None_specified"),
    "timestamp": meta["params"].get("timestamp_utc","None_specified")
}


git_url = f"{BASE}/{TABLES['git_metadata']}/data"

response = requests.post(
    git_url,
    headers=headers,
    auth=auth,
    json={"data": git_payload}
)

# --- Logging ---
print("\n🔍 Git Metadata POST")
print("➡️ URL:", git_url)
print("📦 Payload:")
print(json.dumps(git_payload, indent=2))
print("📤 Status Code:", response.status_code)
print("📝 Response Text:", response.text)

# --- Build payload dynamically ---
dataset_payload = {
    "dataset_id": dataset_id,
    "table_name": meta["tags"].get("dataset_name"),
    "detailed_type": "CSV",
    "classes": int(meta["tags"].get("dataset_num_classes", 3)),
    "features": int(meta["params"]["final_num_features"]),
    "output_type": meta["tags"].get("output_type", "categorical"),
    "version": meta["tags"].get("dataset_version","None_specified"),
    "source_url": meta["tags"].get("DOI_dataset_url","None_specified"),
    "title": meta["tags"].get("DOI_dataset_title","None_specified"),
    "description": meta["tags"].get("DOI_dataset_description", "—"),
    "license": meta["tags"].get("DOI_dataset_license", "—"),
    "creator": meta["tags"].get("DOI_dataset_creator", "—"),
    "created": meta["tags"].get("DOI_dataset_created", "—"),
    "updated": meta["tags"].get("DOI_dataset_updated", "—")
}


# # --- Build URL ---
dataset_url = f"{BASE}/{TABLES['dataset_metadata']}/data"

# --- Send POST request ---
response = requests.post(
    dataset_url,
    headers=headers,
    auth=auth,
    json={"data": dataset_payload}
)

# --- Log request/response ---
print("\n🔍 Dataset Metadata POST")
print("➡️ URL:", dataset_url)
print("📦 Payload:")
print(json.dumps(dataset_payload, indent=2))
print("📤 Status Code:", response.status_code)
print("📝 Response Text:", response.text)


# -- Log POST request for model metadata --
model_payload = {
    "model_id": model_id,
    "name": model_name,
    "algo": estimator,
    "architecture": meta["tags"].get("model_architecture"),
    "features": meta["params"].get("final_feature_names"),
    "label_snap": meta["tags"].get("target_variable_encoded"),
    "train_split": float(meta["params"]["n_train_samples"]) / int(meta["params"]["input_row_count"]),
    "test_split": float(meta["params"]["n_test_samples"]) / int(meta["params"]["input_row_count"]),
    "target_var": meta["tags"].get("target_variable"),
    "label_map": meta["tags"].get("label_map"),
    "feature_select": meta["tags"].get("feature_columns"),
    "imbalance_ratio": meta["tags"].get("imbalance_ratio", 1.0),
    "version": meta["tags"].get("model_version"),
    "serialization_format": meta["tags"].get("model_serialization"),
    "model_path": meta["tags"].get("model_path"),
    "model_short_name": meta["tags"].get("selected_model"),
    "hyperparameters": meta["tags"].get("hyperparameters"),
    "preprocessing_info": meta["tags"].get("preprocessing_info")
}


model_url = f"{BASE}/{TABLES['model_metadata']}/data"
response = requests.post(
    model_url,
    headers=headers,
    auth=auth,
    json={"data": model_payload}
)

# --- Logging ---
print("\n🔍 MODEL METADATA POST")
print("➡️ URL:", model_url)
print("📦 Payload:")
print(json.dumps(model_payload, indent=2))
print("📤 Status Code:", response.status_code)
print("📝 Response Text:", response.text)

# --- Extract Metrics + Justification (final section) ------------------------------------
# --- Allowed lowercase fields from your DB schema ---
allowed_fields = {
    "run_id", "accuracy", "f1_macro", "num_deleted_rows", "num_inserted_rows",
    "precision_macro", "recall_macro", "roc_auc", "row_count_end", "row_count_start",
    "training_accuracy_score", "training_f1_score", "training_log_loss",
    "training_precision_score", "training_recall_score", "training_roc_auc", "training_score",
    "justification_bootstrap", "justification_class_weight", "justification_criterion",
    "justification_dataset_version", "justification_drop_column_x", "justification_ethical_considerations",
    "justification_experiment_name", "justification_intended_use", "justification_max_depth",
    "justification_max_features", "justification_metric_choice", "justification_min_samples_leaf",
    "justification_min_samples_split", "justification_model_choice", "justification_model_limitations",
    "justification_not_intended_for", "justification_n_estimators", "justification_n_jobs",
    "justification_oob_score", "justification_target_variable", "justification_test_split",
    "justification_threshold_accuracy", "justification_verbose"
}

# --- Build raw payload from JSON ---
raw_payload = {
    "run_id": meta.get("run_id", "unknown_run"),
    **meta.get("metrics", {}),
    **{
        k: v for k, v in meta.get("tags", {}).items()
        if k.startswith("justification_") or k.startswith("justification_MLSEA_")
    }
}

# --- Normalize keys and filter only valid ones ---
clean_payload = {
    k.lower(): v for k, v in raw_payload.items()
    if k.lower() in allowed_fields
}

# --- POST request ---
metrics_url = f"{BASE}/{TABLES['metrics_justification']}/data"
response = requests.post(
    metrics_url,
    headers=headers,
    auth=auth,
    json={"data": clean_payload}
)

# --- Log Result ---
print("\n🔍 Cleaned Metrics + Justification POST")
print("➡️ URL:", metrics_url)
print("📦 Payload:")
print(json.dumps(clean_payload, indent=2))
print("📤 Status Code:", response.status_code)
print("📝 Response Text:", response.text)


🔍 Session Metadata POST
➡️ URL: http://localhost/api/database/633f5987-d116-42e8-97fc-36b9c25ade24/table/0eec52d1-3cd0-4c5f-ab61-a7387dbe1449/data
📦 Payload:
{
  "session_id": "51ea707f-6b0e-45b1-8458-c100c05b879d",
  "username": "reema",
  "timestamp": "2025-05-19T10:22:50.772189",
  "hostname": "Purplish",
  "platform": "Windows",
  "python_version": "3.11.5",
  "os_platform": "Windows 10",
  "role": "researcher",
  "project_id": "provo",
  "script_name": "None_specified"
}
📤 Status Code: 201
📝 Response Text: 

🔍 Experiment Metadata POST
➡️ URL: http://localhost/api/database/633f5987-d116-42e8-97fc-36b9c25ade24/table/35d83c73-9a63-4f95-950a-4af699f8c16a/data
📦 Payload:
{
  "runid": "a6da5849dca64ecaad141a1702b89306",
  "sessionid": "51ea707f-6b0e-45b1-8458-c100c05b879d",
  "modelid": "model_iris",
  "datasetid": "519eb3fc-687c-4791-aa13-96d5bee8cbad",
  "git_commit": "c53df1658851cb9c02469abf06ede573625ef73e",
  "invenioid": "10.24432/C56C76",
  "timestamp": "2025-05-19T10:22:50.772

In [ ]:
fetch table data:for export

In [69]:
import requests
import json

DB_ID = "633f5987-d116-42e8-97fc-36b9c25ade24"
HEADERS = {"Accept": "application/json"}
TABLES = {
   
    "experiment_metadata": "35d83c73-9a63-4f95-950a-4af699f8c16a",
    "git_metadata": "09832028-c343-4bb1-82c0-440a78e78c78",
    "dataset_metadata": "79167d03-9afa-45dd-98aa-46647c021f9f",
    "model_metadata": "9d49d172-df6f-4aa2-9141-cb18865138f6",
    "metrics_justification": "e553f49e-1074-4642-b27c-39df9464c777",
    "session_metadata": "0eec52d1-3cd0-4c5f-ab61-a7387dbe1449",
}

run_id = run_id
table_objects = {}

for table, table_id in TABLES.items():
    url = f"http://localhost/api/database/{DB_ID}/table/{table_id}/data?size=100000&page=0"
    try:
        r = requests.get(url, headers=HEADERS)
        r.raise_for_status()
        records = r.json()

        if table == "experiment_metadata":
            match = next((x for x in records if x.get("runid") == run_id), None)
        elif table == "metrics_justification":
            match = next((x for x in records if x.get("run_id") == run_id), None)
        elif table == "model_metadata":
            mid = table_objects.get("experiment_metadata", {}).get("modelid")
            match = next((x for x in records if x.get("model_id") == mid), None)
        elif table == "dataset_metadata":
            did = table_objects.get("experiment_metadata", {}).get("datasetid")
            match = next((x for x in records if x.get("dataset_id") == did), None)
        elif table == "session_metadata":
            sid = table_objects.get("experiment_metadata", {}).get("sessionid")
            match = next((x for x in records if x.get("session_id") == sid), None)
        elif table == "git_metadata":
            ghash = table_objects.get("experiment_metadata", {}).get("git_commit")
            match = next((x for x in records if x.get("commit_hash") == ghash), None)
        else:
            match = None

        table_objects[table] = match or f"❌ No match for {table}"

    except Exception as e:
        table_objects[table] = f"⚠️ Request failed: {e}"

# Print results
print(json.dumps(table_objects, indent=2))


{
  "experiment_metadata": {
    "runid": "a6da5849dca64ecaad141a1702b89306",
    "sessionid": "51ea707f-6b0e-45b1-8458-c100c05b879d",
    "modelid": "model_iris",
    "datasetid": "519eb3fc-687c-4791-aa13-96d5bee8cbad",
    "git_commit": "c53df1658851cb9c02469abf06ede573625ef73e",
    "invenioid": "10.24432/C56C76",
    "timestamp": "2025-05-19T10:22:50.772189",
    "run_name": "legendary-pug-966",
    "experiment_id": "None_specified",
    "training_start_time": "2025-05-19T12:23:04.499804",
    "training_end_time": "2025-05-19T12:23:21.538475",
    "source_file": "C:\\Users\\reema\\AppData\\Roaming\\Python\\Python311\\site-packages\\ipykernel_launcher.py",
    "source_notebook": "RQ1_2.ipynb"
  },
  "git_metadata": {
    "commit_hash": "c53df1658851cb9c02469abf06ede573625ef73e",
    "repo_url": "https://archive.ics.uci.edu/dataset/53",
    "branch": "main",
    "author": "Reema George",
    "author_email": "106236154+reema-dass26@users.noreply.github.com",
    "version": "v12",
    

In [71]:
!pip install rdflib 

In [79]:
# Fixing the issue by ensuring the key exists in the provided data structure
# Reloading the full table_objects structure with metrics_justification included
from rdflib import Graph, Literal, RDF, URIRef, Namespace
from rdflib.namespace import XSD, FOAF, PROV
import json
# === Helper ===
def safe_uri(base, value, suffix=""):
    """Return a safe URIRef or None if value is invalid."""
    if not value or str(value).lower() in ["none", "not available", "—"]:
        return None
    cleaned = str(value).strip().replace(" ", "_").replace("/", "_").replace(":", "_")
    return URIRef(base + cleaned + suffix)

def safe_add_literal(graph, subject, predicate_ns, key, value):
    """Only add a literal if value is valid."""
    if value is not None and str(value).lower() not in ["none", "not available", "—"]:
        graph.add((subject, predicate_ns[key], Literal(value)))

## Namespaces
EX = Namespace("http://example.org/")
g = Graph()
g.bind("prov", PROV)
g.bind("ex", EX)

# Extract entities
experiment = table_objects["experiment_metadata"]
model = table_objects["model_metadata"]
dataset = table_objects["dataset_metadata"]
git = table_objects["git_metadata"]
session = table_objects["session_metadata"]
metrics = table_objects["metrics_justification"]

# === Create safe URIs (fallback to blank node if invalid) ===
model_uri = safe_uri(EX, f"model/{model.get('model_id')}") or BNode()
dataset_uri = safe_uri(EX, f"dataset/{experiment.get('datasetid')}") or BNode()
code_uri = safe_uri(EX, f"code/{git.get('commit_hash')}") or BNode()
activity_uri = safe_uri(EX, f"run/{experiment.get('runid')}") or BNode()
agent_uri = safe_uri(EX, f"agent/{session.get('username')}") or BNode()

# === Base RDF types ===
g.add((model_uri, RDF.type, PROV.Entity))
g.add((dataset_uri, RDF.type, PROV.Entity))
g.add((code_uri, RDF.type, PROV.Entity))
g.add((activity_uri, RDF.type, PROV.Activity))
g.add((agent_uri, RDF.type, PROV.Agent))

# === Agent name ===
safe_add_literal(g, agent_uri, FOAF, "name", session.get("username"))

# === Provenance relationships ===
safe_add_literal(g, activity_uri, PROV, "startedAtTime", experiment.get("training_start_time"))
safe_add_literal(g, activity_uri, PROV, "endedAtTime", experiment.get("training_end_time"))
g.add((activity_uri, PROV.used, dataset_uri))
g.add((activity_uri, PROV.used, code_uri))
g.add((model_uri, PROV.wasGeneratedBy, activity_uri))
g.add((activity_uri, PROV.wasAssociatedWith, agent_uri))

# === Attach all valid literals ===
for data, uri in [
    (model, model_uri),
    (dataset, dataset_uri),
    (git, code_uri),
    (experiment, activity_uri),
    (session, agent_uri),
    (metrics, activity_uri)
]:
    for k, v in data.items():
        safe_add_literal(g, uri, EX, k, v)

# === Output ===
jsonld_output = g.serialize(format="json-ld", indent=2)
with open(f"{summary_dir}/prov_JSONLD_export.jsonld", "w", encoding="utf-8") as f:
    f.write(jsonld_output)

rdfxml_output = g.serialize(format="xml", encoding="utf-8")
with open(f"{summary_dir}/prov_RDFXML_export.rdf", "wb") as f:
    f.write(rdfxml_output)

print("✅ Provenance RDF + JSON-LD export completed.")

✅ Provenance RDF + JSON-LD export completed.


In [80]:
from rdflib import Graph, Literal, RDF, URIRef, Namespace
from rdflib.namespace import XSD, FOAF, PROV
from collections import defaultdict
import json

# Define namespaces
EX = Namespace("http://example.org/")
g = Graph()
g.bind("prov", PROV)
g.bind("ex", EX)

# -- Safe helpers --
def safe_literal(value):
    return (
        value
        and str(value).strip().lower() not in ["none", "no justification provided", "—"]
    )

def safe_uri(base, value, suffix=""):
    if not safe_literal(value):
        return None
    cleaned = str(value).strip().replace(" ", "_").replace("/", "_").replace(":", "_")
    return URIRef(base + cleaned + suffix)

# Extract entities
experiment = table_objects["experiment_metadata"]
model = table_objects["model_metadata"]
dataset = table_objects["dataset_metadata"]
git = table_objects["git_metadata"]
session = table_objects["session_metadata"]
metrics = table_objects["metrics_justification"]

# Create URIs
model_uri = safe_uri(EX, f"model/{model.get('model_id')}") or URIRef(EX["model/default"])
dataset_uri = safe_uri(EX, f"dataset/{experiment.get('datasetid')}") or URIRef(EX["dataset/default"])
code_uri = safe_uri(EX, f"code/{git.get('commit_hash')}") or URIRef(EX["code/default"])
activity_uri = safe_uri(EX, f"run/{experiment.get('runid')}") or URIRef(EX["run/default"])
agent_uri = safe_uri(EX, f"agent/{session.get('username')}") or URIRef(EX["agent/default"])

# Add core RDF types
g.add((model_uri, RDF.type, PROV.Entity))
g.add((dataset_uri, RDF.type, PROV.Entity))
g.add((code_uri, RDF.type, PROV.Entity))
g.add((activity_uri, RDF.type, PROV.Activity))
g.add((agent_uri, RDF.type, PROV.Agent))
g.add((agent_uri, FOAF.name, Literal(session.get("username"))))

# Add PROV relationships
if safe_literal(experiment.get("training_start_time")):
    g.add((activity_uri, PROV.startedAtTime, Literal(experiment["training_start_time"], datatype=XSD.dateTime)))
if safe_literal(experiment.get("training_end_time")):
    g.add((activity_uri, PROV.endedAtTime, Literal(experiment["training_end_time"], datatype=XSD.dateTime)))

g.add((activity_uri, PROV.used, dataset_uri))
g.add((activity_uri, PROV.used, code_uri))
g.add((model_uri, PROV.wasGeneratedBy, activity_uri))
g.add((activity_uri, PROV.wasAssociatedWith, agent_uri))

# Attach literals from each metadata block
for data, uri in [
    (model, model_uri),
    (dataset, dataset_uri),
    (git, code_uri),
    (experiment, activity_uri),
    (session, agent_uri),
    (metrics, activity_uri)
]:
    for k, v in data.items():
        if safe_literal(v):
            g.add((uri, EX[k], Literal(v)))

# Convert RDF triples into normal JSON structure
prov_json = defaultdict(dict)
for subj, pred, obj in g:
    subj_str = str(subj)
    pred_str = str(pred)
    obj_str = str(obj)

    if pred_str not in prov_json[subj_str]:
        prov_json[subj_str][pred_str] = obj_str
    else:
        current = prov_json[subj_str][pred_str]
        if isinstance(current, list):
            current.append(obj_str)
        else:
            prov_json[subj_str][pred_str] = [current, obj_str]

# Save to disk
output_path = f"{summary_dir}/prov_JSON_export.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(prov_json, f, indent=2)

print(f"✅ JSON file written to: {output_path}")


✅ JSON file written to: C:\Users\reema\REPO\notebooks\RQ_notebooks\MODEL_PROVENANCE\RandomForest_Iris_v20250519_122304/prov_JSON_export.json


########################################################################
# Post metadata to INVENIO
########################################################################

In [ ]:

# -----------------------------------------------------------------------------
# Configuration
# -----------------------------------------------------------------------------
API_BASE   = "https://127.0.0.1:5000"
TOKEN      = "8LnqJuz3TsBHffnDJ3isPLHYHtRbWrC0M667Nb5haEbnXpWqGbFRyfDApymr"
VERIFY_SSL = False  # only for self‐signed dev

HEADERS_JSON = {
    "Accept":        "application/json",
    "Content-Type":  "application/json",
    "Authorization": f"Bearer {TOKEN}",
}

HEADERS_OCTET = {
    "Content-Type":  "application/octet-stream",
    "Authorization": f"Bearer {TOKEN}",
}

# The folders you want to walk & upload:
TO_UPLOAD = ["Trained_models", "plots", "MODEL_PROVENANCE"]


# -----------------------------------------------------------------------------
# 1) Create draft with ALL required metadata
# -----------------------------------------------------------------------------
def create_draft():
    payload = {
  "metadata": {
    "title":            "RandomForest Iris Model Artifacts",
    "creators": [ {
      "person_or_org": {
        "type":        "personal",
        "given_name":  "Reema",
        "family_name": "Dass"
      }
    } ],
    "publication_date": "2025-04-24",
    "resource_type":    { "id": "software" },
    "access": {
      "record": "public",
      "files":  "public"
    }
  }
}
    r = requests.post(f"{API_BASE}/api/records",
                      headers=HEADERS_JSON,
                      json=payload,
                      verify=VERIFY_SSL)
    r.raise_for_status()
    draft = r.json()
    print("✅ Draft created:", draft["id"])
    return draft["id"], draft["links"]


# -----------------------------------------------------------------------------
# 2) Register, upload and commit a single file
# -----------------------------------------------------------------------------
def upload_and_commit(links, key, path):
    # 2a) register the filename in the draft
    r1 = requests.post(links["files"],
                       headers=HEADERS_JSON,
                       json=[{"key": key}],
                       verify=VERIFY_SSL)
    r1.raise_for_status()
    entry = next(e for e in r1.json()["entries"] if e["key"] == key)
    file_links = entry["links"]

    # 2b) upload the bytes
    with open(path, "rb") as fp:
        r2 = requests.put(file_links["content"],
                          headers=HEADERS_OCTET,
                          data=fp,
                          verify=VERIFY_SSL)
    r2.raise_for_status()

    # 2c) commit the upload
    r3 = requests.post(file_links["commit"],
                       headers=HEADERS_JSON,
                       verify=VERIFY_SSL)
    r3.raise_for_status()
    print(f"  • Uploaded {key}")


# -----------------------------------------------------------------------------
# 3) Walk each folder and upload every file
# -----------------------------------------------------------------------------
def upload_folder(links):
    for folder in TO_UPLOAD:
        if not os.path.isdir(folder):
            print(f"⚠️ Skipping missing folder {folder}")
            continue
        base = os.path.dirname(folder) or folder
        for root, _, files in os.walk(folder):
            for fn in files:
                local = os.path.join(root, fn)
                # create a POSIX‐style key preserving subfolders
                key = os.path.relpath(local, start=base).replace(os.sep, "/")
                upload_and_commit(links, key, local)


# -----------------------------------------------------------------------------
# 4) Publish the draft
# -----------------------------------------------------------------------------
def publish(links):
    r = requests.post(links["publish"],
                      headers=HEADERS_JSON,
                      verify=VERIFY_SSL)
    if not r.ok:
        print("❌ Publish failed:", r.status_code, r.text)
        try: print(r.json())
        except: pass
        r.raise_for_status()
    print("✅ Published:", r.json()["id"])


# -----------------------------------------------------------------------------
# Main
# -----------------------------------------------------------------------------
if __name__ == "__main__":
    recid, links = create_draft()
    upload_folder(links)
    publish(links)

  


########################################################################
# FETCH metadata from INVENIO
########################################################################

In [ ]:
def fetch_metadata(record_id, model_name, api_base, headers, verify_ssl=True):
    """
    Fetch Invenio metadata and save to a file named after the model inside 'Invenio_metadata' folder.
    """
    response = requests.get(f"{API_BASE}/api/records/{record_id}",
                            headers=headers,
                            verify=VERIFY_SSL)
    response.raise_for_status()
    metadata = response.json()

    print("✅ Metadata fetched successfully")

    # Create the folder if it doesn't exist
    os.makedirs("Invenio_metadata", exist_ok=True)

    # Construct path and save
    file_path = os.path.join("Invenio_metadata", f"{model_name}_invenio.json")
    with open(file_path, "w") as f:
        json.dump(metadata, f, indent=4)

    print(f"✅ Metadata saved at {file_path}")
    return file_path
path = fetch_metadata(recid, model_name, api_base=API_BASE, headers=HEADERS_JSON)
print(path)

METADATA EXTRACTION FROM INVENIO and ADD it to main Provenence FILE

In [ ]:
# ----------------------------
# Function: Extract metadata
# ----------------------------
def extract_metadata(metadata):
    print("✅ Metadata loaded successfully")
    print("ℹ️ ID:", metadata.get("id", "N/A"))
    print("🔍 Extracting required fields...")

    extracted_data = {
        "invenio_metadata": {
            "id": metadata.get("id", ""),
            "title": metadata.get("metadata", {}).get("title", ""),
            "creator": ", ".join([
                creator["person_or_org"].get("name", "")
                for creator in metadata.get("metadata", {}).get("creators", [])
            ]),
            "publication_date": metadata.get("metadata", {}).get("publication_date", ""),
            "files": [],
            "pids": metadata.get("pids", {}),
            "version_info": metadata.get("versions", {}),
            "status": metadata.get("status", ""),
            "views": metadata.get("stats", {}).get("this_version", {}).get("views", 0),
            "downloads": metadata.get("stats", {}).get("this_version", {}).get("downloads", 0),
        }
    }

    for key, file_info in metadata.get("files", {}).get("entries", {}).items():
        file_detail = {
            "key": key,
            "url": file_info["links"].get("content", ""),
            "size": file_info.get("size", 0),
            "mimetype": file_info.get("mimetype", ""),
            "checksum": file_info.get("checksum", ""),
            "metadata": file_info.get("metadata", {}),
        }
        extracted_data["invenio_metadata"]["files"].append(file_detail)

    return extracted_data


invenio_path = f"Invenio_metadata/{model_name}_invenio.json"
run_summary_path = f"MODEL_PROVENANCE/{model_name}/{model_name}_run_summary.json"

# ----------------------------
# Step 1: Load Invenio metadata
# ----------------------------
with open(invenio_path, "r") as f:
    original_metadata = json.load(f)

# ----------------------------
# Step 2: Extract metadata
# ----------------------------
extracted_metadata = extract_metadata(original_metadata)
print("📤 Extracted Metadata Preview:")
print(json.dumps(extracted_metadata, indent=4)[:1000])  # Preview

# ----------------------------
# Step 3: Load run summary
# ----------------------------
with open(run_summary_path, "r") as f:
    existing_metadata = json.load(f)

# ----------------------------
# Step 4: Merge metadata
# ----------------------------
existing_metadata.update(extracted_metadata)

# ----------------------------
# Step 5: Save updated summary
# ----------------------------
with open(run_summary_path, "w") as f:
    json.dump(existing_metadata, f, indent=4)

print(f"✅ Invenio metadata embedded successfully into: {run_summary_path}")